<a href="https://colab.research.google.com/github/armahdavi/ML-xgboost-regressor---rapid-filter-forensics_rff-dust-recovery-from-HVAC-filter/blob/main/lt_package_pyspark_practive.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install PySpark
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488490 sha256=ad08113baa022ca181f06d53ecf6562fdcbe7168a54bd07177b7a68546625e44
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [57]:
# Setup PySpark in Google Colab
from pyspark.sql import SparkSession
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [73]:
# Define the file path in your Google Drive
file_path = '/content/drive/My Drive/lt_complete_weekly_sum.csv'

spark = SparkSession.builder.getOrCreate()
# Read the CSV file into a PySpark DataFrame
df = spark.read.csv(file_path, header=True)
col_list = df.columns

df = df.withColumn("v", lit(0.1))
df = df.withColumn("met", lit(1.0))
df = df.withColumn("clo", lit(0.5))


# selected_cols = ['GMTtime'] + col_list[col_list.index('tmp'):col_list.index('PPD_10')]
df = df.select('tmp', 'mrt', 'v', 'rhm', 'met', 'clo')
print(type(df))
df.show(n = 100)


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


<class 'pyspark.sql.dataframe.DataFrame'>
+------+------+---+------+---+---+
|   tmp|   mrt|  v|   rhm|met|clo|
+------+------+---+------+---+---+
|28.742|27.924|0.1|17.576|1.0|0.5|
|28.667| 27.85|0.1|17.219|1.0|0.5|
|28.642|27.899|0.1|17.147|1.0|0.5|
|28.642|27.875|0.1|16.864|1.0|0.5|
|28.642|27.825|0.1|16.476|1.0|0.5|
|28.593|27.825|0.1|16.225|1.0|0.5|
|28.543|27.751|0.1|16.152|1.0|0.5|
|28.518|27.727|0.1|16.222|1.0|0.5|
|28.493|27.751|0.1| 16.22|1.0|0.5|
|28.468|27.653|0.1|15.865|1.0|0.5|
|28.444|27.628|0.1|15.723|1.0|0.5|
|28.419|27.628|0.1|15.651|1.0|0.5|
|28.369|27.604|0.1|15.577|1.0|0.5|
|28.345| 27.53|0.1|15.576|1.0|0.5|
| 28.32|27.554|0.1|15.504|1.0|0.5|
| 28.32|27.554|0.1|15.504|1.0|0.5|
|28.295|27.481|0.1|15.148|1.0|0.5|
|28.245|27.431|0.1|15.146|1.0|0.5|
|28.221|27.456|0.1|15.216|1.0|0.5|
|28.171|27.382|0.1|15.284|1.0|0.5|
|28.171|27.407|0.1|15.249|1.0|0.5|
|28.171|27.382|0.1|15.249|1.0|0.5|
|28.147|27.308|0.1|14.929|1.0|0.5|
|28.072|27.259|0.1|15.634|1.0|0.5|
|27.998|27.18

In [9]:
!pip install pythermalcomfort

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.0/128.0 kB 6.6 MB/s eta 0:00:00


In [75]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
from pyspark.sql.types import StructType, StructField, FloatType
import pythermalcomfort as pt
from pyspark.sql.functions import lit

# Define the UDF to calculate PMV and PPD
def calculate_pmv_ppd(ta, tr, v, rh, met, clo):
    pmv, ppd = pt.pmv_ppd(ta, tr, v, rh, met, clo).values()
    return pmv, ppd

# Register the UDF
pmv_ppd_udf = udf(calculate_pmv_ppd, StructType([
    StructField("pmv", FloatType(), True),
    StructField("ppd", FloatType(), True)
]))

# Access individual fields from the StructType column
df = df.withColumn("pmv", pmv_ppd_udf(df['tmp'], df['mrt'], df['v'], df['rhm'], df['met'], df['clo']))
df = df.withColumn("ppd", pmv_ppd_udf(df['tmp'], df['mrt'], df['v'], df['rhm'], df['met'], df['clo']))

df = df.select('tmp', 'mrt', 'v', 'rhm', 'met', 'clo', 'pmv', 'ppd')


print(df.count())
df.take(10)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


3881524


PythonException: 
  An exception was thrown from the Python worker. Please see the stack trace below.
Traceback (most recent call last):
  File "<ipython-input-75-e75ef0d2cf88>", line 9, in calculate_pmv_ppd
  File "/usr/local/lib/python3.10/dist-packages/pythermalcomfort/models/pmv_ppd.py", line 177, in pmv_ppd
    ) = check_standard_compliance_array(
  File "/usr/local/lib/python3.10/dist-packages/pythermalcomfort/utilities.py", line 173, in check_standard_compliance_array
    tdb_valid = valid_range(params["tdb"], (10.0, 30.0))
  File "/usr/local/lib/python3.10/dist-packages/pythermalcomfort/shared_functions.py", line 6, in valid_range
    return np.where((x >= valid[0]) & (x <= valid[1]), x, np.nan)
numpy.core._exceptions._UFuncNoLoopError: ufunc 'greater_equal' did not contain a loop with signature matching types (<class 'numpy.dtypes.StrDType'>, <class 'numpy.dtypes.Float64DType'>) -> None


In [53]:


# Create a Spark session
spark = SparkSession.builder.appName("PMV_PPD_Calculator").getOrCreate()



df.take(10)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


PythonException: 
  An exception was thrown from the Python worker. Please see the stack trace below.
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 1247, in main
    process()
  File "/usr/local/lib/python3.10/dist-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 1239, in process
    serializer.dump_stream(out_iter, outfile)
  File "/usr/local/lib/python3.10/dist-packages/pyspark/python/lib/pyspark.zip/pyspark/serializers.py", line 225, in dump_stream
    self.serializer.dump_stream(self._batched(iterator), stream)
  File "/usr/local/lib/python3.10/dist-packages/pyspark/python/lib/pyspark.zip/pyspark/serializers.py", line 146, in dump_stream
    for obj in iterator:
  File "/usr/local/lib/python3.10/dist-packages/pyspark/python/lib/pyspark.zip/pyspark/serializers.py", line 214, in _batched
    for item in iterator:
  File "/usr/local/lib/python3.10/dist-packages/pyspark/python/l